In [1]:
import pandas as pd
import numpy as np

In [2]:
medical_df = pd.read_csv("subset_medical_data.csv")
print(medical_df.shape)
medical_df.head()

(7896832, 13)


,Member Life ID,Gender Code,Claim Type Code,Claim Number,Claim Disposition,Current Procedural Terminology,Revenue Codes,Primary Diagnosis Code-ICD10,ICD10 Surgical Procedure Code 1,Line Service From Date,Line Service thru Date,Billed Amount,Paid Amount
0,3269408,M,P,831210364500,1,97014,*,M9904,*,2018-10-23,2018-10-23,28.0,0.00
1,3269408,M,P,835410338700,1,98941,*,S335XXA,*,2018-11-13,2018-11-13,50.0,5.23
2,3269408,M,P,835410338700,1,97140,*,S335XXA,*,2018-11-13,2018-11-13,45.0,18.14
3,3269408,M,P,835410860300,1,97012,*,S335XXA,*,2018-12-04,2018-12-04,55.0,9.84
4,3269408,M,P,835410860300,1,97014,*,S335XXA,*,2018-12-04,2018-12-04,28.0,9.83


In [3]:
medical_df.columns = [x.replace(" ", "_").lower() for x in medical_df.columns]

In [4]:
medical_df.nunique()

member_life_id                       99999
gender_code                              3
claim_type_code                          4
claim_number                       2858060
claim_disposition                        4
current_procedural_terminology       10109
revenue_codes                          363
primary_diagnosis_code-icd10         20040
icd10_surgical_procedure_code_1       1912
line_service_from_date                 730
line_service_thru_date                 756
billed_amount                       190868
paid_amount                         180207
dtype: int64

In [5]:
pharmacy_df = pd.read_csv("subset_pharmacy_data.csv")
pharmacy_df.columns = [x.replace(" ", "_").lower() for x in pharmacy_df.columns]
print(pharmacy_df.shape)
pharmacy_df.head()

(1512707, 11)


,member_life_id,birth_date,gender_code,claim_number,ndc,drug_name,drug_tier,fill_date,paid_date,billed_amount,paid_amount
0,351051,1998-02-26,M,170056591632164999,68382048428,MESALAMINE TAB 800MG DR,Other,2017-01-05,2017-01-05,0.00,0.0
1,351051,1998-02-26,M,170176769349106999,74433902,HUMIRA PEN KIT 40MG/0.8,PREF BR NM DRGS HIGHER CPY,2017-01-17,2017-01-20,-8608.84,-150.0
2,351051,1998-02-26,M,170353564327096999,74433906,HUMIRA PEN KIT CROHNS,Other,2017-02-04,2017-02-04,0.00,0.0
3,351051,1998-02-26,M,170406056167191997,74433906,HUMIRA PEN KIT CROHNS,Other,2017-02-09,2017-02-09,0.00,0.0
4,351051,1998-02-26,M,170412764402041997,74433906,HUMIRA PEN KIT CROHNS,Other,2017-02-10,2017-02-10,0.00,0.0


In [6]:
pharmacy_df.rename(columns={"billed_amount": "billed_amount_pharm", 
                            "paid_amount": "paid_amount_pharm",
                           "claim_number": "claim_number_pharm"}, inplace=True)

In [7]:
pharmacy_df.nunique()

member_life_id           54000
birth_date               22477
gender_code                  3
claim_number_pharm     1388559
ndc                      20790
drug_name                 6938
drug_tier                    8
fill_date                  730
paid_date                  878
billed_amount_pharm      92455
paid_amount_pharm        20524
dtype: int64

In [8]:
medical_df['line_service_thru_date'] = pd.to_datetime(medical_df['line_service_thru_date'])
pharmacy_df['fill_date'] = pd.to_datetime(pharmacy_df['fill_date'])

In [9]:
medical_df = medical_df[(medical_df['line_service_thru_date']>=pd.to_datetime("2018-01-01")) & 
           (medical_df['line_service_thru_date']<=pd.to_datetime("2018-12-31"))]

pharmacy_df = pharmacy_df[(pharmacy_df['fill_date']>=pd.to_datetime("2018-01-01")) & 
           (pharmacy_df['fill_date']<=pd.to_datetime("2018-12-31"))]

print(medical_df.shape)
print(pharmacy_df.shape)

(4140178, 13)
(733739, 11)


In [10]:
from sqlite3 import connect

conn = connect(':memory:')

medical_df.to_sql(name='medical_data', index=False, con=conn)
pharmacy_df.to_sql(name='pharmacy_data', index=False, con=conn)

733739

In [11]:
query = """WITH merged_data AS (
    SELECT
        m.*,
        p.*,
        p.fill_date - m.line_service_thru_date AS date_difference
    FROM
        medical_data m
        INNER JOIN pharmacy_data p ON m.member_life_id = p.member_life_id
    WHERE
        p.fill_date >= m.line_service_thru_date
)

SELECT
    *
FROM (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY member_life_id, line_service_thru_date ORDER BY date_difference, billed_amount_pharm) AS rn
    FROM
        merged_data
) t
WHERE
    rn = 1;
"""

merged_data = pd.read_sql(query, conn)

In [12]:
merged_data.shape

(453005, 26)

In [13]:
merged_data.head()

,member_life_id,gender_code,claim_type_code,claim_number,claim_disposition,current_procedural_terminology,revenue_codes,primary_diagnosis_code-icd10,icd10_surgical_procedure_code_1,line_service_from_date,...,claim_number_pharm,ndc,drug_name,drug_tier,fill_date,paid_date,billed_amount_pharm,paid_amount_pharm,date_difference,rn
0,169,F,P,805119522400,1,97161,*,M4692,*,2018-01-23,...,181553106521188999,781552810,BUPROPN HCL TAB 150MG XL,GENERIC DRUGS LOWEST CPY,2018-06-04 00:00:00,2018-06-04,49.88,30.9,0,1
1,169,F,P,805119521800,1,97112,*,M4692,*,2018-01-26,...,181553106521188999,781552810,BUPROPN HCL TAB 150MG XL,GENERIC DRUGS LOWEST CPY,2018-06-04 00:00:00,2018-06-04,49.88,30.9,0,1
2,169,F,P,805119522800,1,97530,*,M4692,*,2018-01-29,...,181553106521188999,781552810,BUPROPN HCL TAB 150MG XL,GENERIC DRUGS LOWEST CPY,2018-06-04 00:00:00,2018-06-04,49.88,30.9,0,1
3,169,F,P,803313780600,1,90686,*,Z23,*,2018-01-30,...,181553106521188999,781552810,BUPROPN HCL TAB 150MG XL,GENERIC DRUGS LOWEST CPY,2018-06-04 00:00:00,2018-06-04,49.88,30.9,0,1
4,169,F,P,805119520100,1,97112,*,M4692,*,2018-02-06,...,181553106521188999,781552810,BUPROPN HCL TAB 150MG XL,GENERIC DRUGS LOWEST CPY,2018-06-04 00:00:00,2018-06-04,49.88,30.9,0,1


In [14]:
merged_data.to_csv("merged_med_pharmacy_2018.csv", index=False)